In [1]:
import numpy as np
import ot
import torch
from math import sqrt

from TheGAN import LevyGAN

In [2]:
config = {
    'device': torch.device('cpu'),
    'ngpu': 0,
    'w dim': 4,
    'a dim': 6,
    'noise size': 62,
    'which generator': 1,
    'which discriminator': 1,
    'generator symmetry mode': 'Hsym',
    'generator last width': 6,
    's dim': 16,
    'leakyReLU slope': 0.2,
    'num epochs': 20,
    'num Chen iters': 5000,
    'optimizer': 'Adam',
    'lrG': 0.0001,
    'lrD': 0.0005,
    'beta1': 0,
    'beta2': 0.99,
    'Lipschitz mode': 'gp',
    'weight clipping limit': 0.01,
    'gp weight': 10.0,
    'batch size': 1024,
    'test batch size': 65536,
    'num tests for 2d': 8,
    'W fixed whole': [1.0, -0.5, -1.2, -0.3, 0.7, 0.2, -0.9, 0.1, 1.7]
}

training_config = {
    'num epochs': 2,
    'num Chen iters': 5000,
    'optimizer': 'Adam',
    'lrG': 0.0001,
    'lrD': 0.0005,
    'beta1': 0,
    'beta2': 0.99,
    'Lipschitz mode': 'gp',
    'weight clipping limit': 0.01,
    'gp weight': 10.0,
    'batch size': 1024,
}

levG = LevyGAN(config)
levG.load_dicts_unstructured('model_saves/GAN1_4d_62noise_min_sum_AMAZING_Hsymgenerator.pt','model_saves/GAN1_4d_62noise_min_sum_AMAZING_Hsymdiscriminator.pt')
levG.classic_train(training_config)

blub


/home/andy/anaconda3/envs/GAN/lib/python3.10/site-packages/ot/lp/solver_1d.py:41: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484746364/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  cws = cws.T.contiguous()


epoch: 0/2, iter: 0, gradient norm: 0.78503, discriminator dist: -14.32564, st_dev error:  0.13414, joint_wass_dist:  0.51445
errs: ['0.0284', '0.0301', '0.0286', '0.0341', '0.0377', '0.0261'], ch_err: ['0.4945', '0.4952', '0.2258', '0.4806', '0.2018', '0.2133'] 
epoch: 0/2, iter: 100, gradient norm: 0.98492, discriminator dist: -13.80314, st_dev error:  0.14148, joint_wass_dist:  0.46446
errs: ['0.0553', '0.0513', '0.0659', '0.0669', '0.0838', '0.0592'], ch_err: ['0.4572', '0.4564', '0.1732', '0.4431', '0.1743', '0.1722'] 
epoch: 0/2, iter: 200, gradient norm: 0.97327, discriminator dist: -14.01418, st_dev error:  0.14289, joint_wass_dist:  0.46718
errs: ['0.0668', '0.0564', '0.0741', '0.0728', '0.0927', '0.0657'], ch_err: ['0.4515', '0.4492', '0.1663', '0.4407', '0.1697', '0.1658'] 
epoch: 0/2, iter: 300, gradient norm: 0.89909, discriminator dist: -14.20497, st_dev error:  0.14338, joint_wass_dist:  0.47277
errs: ['0.0694', '0.0591', '0.0708', '0.0739', '0.0954', '0.0664'], ch_err: 

KeyboardInterrupt: 